In [20]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")

# シングルエージェント

# MCP 対応

In [4]:
import time
import json

from azure.ai.agents.models import MessageTextContent, ListSortOrder
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, MCPToolDefinition, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval

In [23]:
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

mcp_tool = McpTool(
    server_label="MicrosoftDocs",
    server_url="https://learn.microsoft.com/api/mcp",
)

with project_client:    
    agent = project_client.agents.create_agent(
        model="gpt-4.1",
        name="my-mcp-agent",
        instructions=(
            "You are a helpful assistant. "
            "Use the tools provided to answer the user's questions. "
            "Be sure to cite your sources."
        ),
        tools=mcp_tool.definitions
    )
    print(f"Created agent, ID: {agent.id}")
    print(agent.as_dict())

    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content="Azure AI Foundry Agent Service の最新情報は？"
    )
    print(f"Created message, ID: {message.id}")

    
    run = project_client.agents.runs.create(
        thread_id=thread.id,
        agent_id=agent.id
    )


    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                project_client.agents.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    # Retrieve the generated response:
    messages = project_client.agents.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)

    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")


Created agent, ID: asst_WivSY97rID2YcjM7jHftneyU
{'id': 'asst_WivSY97rID2YcjM7jHftneyU', 'object': 'assistant', 'created_at': 1753429679, 'name': 'my-mcp-agent', 'description': None, 'model': 'gpt-4.1', 'instructions': "You are a helpful assistant. Use the tools provided to answer the user's questions. Be sure to cite your sources.", 'tools': [{'type': 'mcp', 'server_label': 'MicrosoftDocs', 'server_url': 'https://learn.microsoft.com/api/mcp', 'allowed_tools': None}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
Created thread, ID: thread_y0pfbF8mffCpxBqsqCWg7S5C
Created message, ID: msg_wpGSn0sXEuJDRUNUseZtNaM2
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Approving tool call: {'id': 'call_0QVffRSimwn3Eu2vqGWHQwqN', 'type': 'mcp', 'arguments': '{"question":"Azure AI Foundry Agent Service æ\x9c\x80æ\x96°æ\x83\x85å\xa0±"}', 'name': 'microsoft_docs_search', 'server_label': 'MicrosoftDocs'}
tool_ap